In [0]:
dataframe = spark.read.table("retail_csv")

dataframe.show()

+----------+----------+--------------------+--------+-------------------+----------+-----------+--------------+
invoice_no|stock_code| description|quantity| invoice_date|unit_price|customer_id| country|
+----------+----------+--------------------+--------+-------------------+----------+-----------+--------------+
 489434| 85048|15CM CHRISTMAS GL...| 12|2009-12-01 07:45:00| 6.95| 13085|United Kingdom|
 489434| 79323P| PINK CHERRY LIGHTS| 12|2009-12-01 07:45:00| 6.75| 13085|United Kingdom|
 489434| 79323W| WHITE CHERRY LIGHTS| 12|2009-12-01 07:45:00| 6.75| 13085|United Kingdom|
 489434| 22041|RECORD FRAME 7" S...| 48|2009-12-01 07:45:00| 2.1| 13085|United Kingdom|
 489434| 21232|STRAWBERRY CERAMI...| 24|2009-12-01 07:45:00| 1.25| 13085|United Kingdom|
 489434| 22064|PINK DOUGHNUT TRI...| 24|2009-12-01 07:45:00| 1.65| 13085|United Kingdom|
 489434| 21871| SAVE THE PLANET MUG| 24|2009-12-01 07:45:00| 1.25| 13085|United Kingdom|
 489434| 21523|FANCY FONT HOME S...| 10|2009-12-01 07:45:00| 5.95| 13085|United Kingdom|
 489435| 22350| CAT BOWL | 12|2009-12-01 07:46:00| 2.55| 13085|United Kingdom|
 489435| 22349|DOG BOWL , CHASIN...| 12|2009-12-01 07:46:00| 3.75| 13085|United Kingdom|
 489435| 22195|HEART MEASURING S...| 24|2009-12-01 07:46:00| 1.65| 13085|United Kingdom|
 489435| 22353|LUNCHBOX WITH CUT...| 12|2009-12-01 07:46:00| 2.55| 13085|United Kingdom|
 489436| 48173C|DOOR MAT BLACK FL...| 10|2009-12-01 09:06:00| 5.95| 13078|United Kingdom|
 489436| 21755|LOVE BUILDING BLO...| 18|2009-12-01 09:06:00| 5.45| 13078|United Kingdom|
 489436| 21754|HOME BUILDING BLO...| 3|2009-12-01 09:06:00| 5.95| 13078|United Kingdom|
 489436| 84879|ASSORTED COLOUR B...| 16|2009-12-01 09:06:00| 1.69| 13078|United Kingdom|
 489436| 22119| PEACE WOODEN BLO...| 3|2009-12-01 09:06:00| 6.95| 13078|United Kingdom|
 489436| 22142|CHRISTMAS CRAFT W...| 12|2009-12-01 09:06:00| 1.45| 13078|United Kingdom|
 489436| 22296|HEART IVORY TRELL...| 12|2009-12-01 09:06:00| 1.65| 13078|United Kingdom|
 489436| 22295|HEART FILIGREE DO...| 12|2009-12-01 09:06:00| 1.65| 13078|United Kingdom|
+----------+----------+--------------------+--------+-------------------+----------+-----------+--------------+
only showing top 20 rows

#Total Invoice Amount

In [0]:
from pyspark.sql.functions import *

total_invoice = dataframe.withColumn('Amount', (col('quantity') * col('unit_price'))).select('invoice_no', 'Amount').groupBy('invoice_no').agg(sum('Amount')).orderBy('invoice_no').withColumnRenamed('sum(Amount)', 'Amount').filter(col('Amount') > 0)
total_invoice = total_invoice.withColumn('Amount', round(total_invoice['Amount'], 2))
total_invoice.show()

+----------+-------+
invoice_no| Amount|
+----------+-------+
 489434| 505.3|
 489435| 145.8|
 489436| 630.33|
 489437| 310.75|
 489438|2286.24|
 489439| 426.3|
 489440| 50.4|
 489441| 344.34|
 489442| 382.37|
 489443| 285.06|
 489444| 141.0|
 489445| 308.44|
 489446| 996.1|
 489447| 130.0|
 489448| 570.24|
 489450| 196.1|
 489460| 485.79|
 489461| 589.33|
 489462| 148.3|
 489465| 1197.8|
+----------+-------+
only showing top 20 rows

#Monthly Placed and Canceled Orders

In [0]:
monthly_placed_orders = dataframe.withColumn('YYYYMM', concat(col('invoice_date').substr(0,4), col('invoice_date').substr(6,2)))
# Get Placed Orders
monthly_placed_orders = dataframe.withColumn('YYYYMM', concat(col('invoice_date').substr(0,4), col('invoice_date').substr(6,2))).groupBy('YYYYMM', 'invoice_no').count().groupBy('YYYYMM').count().withColumnRenamed('Count', 'PlacedBefore')
monthly_placed_orders.show()

+------+------------+
YYYYMM|PlacedBefore|
+------+------------+
201103| 1983|
201001| 1633|
201002| 1969|
201010| 2965|
201009| 2375|
201104| 1744|
201106| 2012|
201102| 1393|
201110| 2637|
201008| 1877|
201107| 1927|
201012| 2025|
201105| 2162|
201109| 2327|
201011| 3669|
201007| 2017|
201111| 3462|
201108| 1737|
201112| 1015|
201004| 1892|
+------+------------+
only showing top 20 rows

In [0]:
# Get cancelled orders
monthly_cancelled_orders = dataframe.withColumn('YYYYMM', concat(col('invoice_date').substr(0,4), col('invoice_date').substr(6,2))).filter(col('invoice_no').startswith("C")).groupBy(col('invoice_no'), col('YYYYMM')).count().groupBy('YYYYMM').count().orderBy(col('YYYYMM')).withColumnRenamed('Count', 'Cancelled')
monthly_cancelled_orders.show()

+------+---------+
YYYYMM|Cancelled|
+------+---------+
200912| 401|
201001| 300|
201002| 240|
201003| 407|
201004| 304|
201005| 407|
201006| 357|
201007| 344|
201008| 273|
201009| 371|
201010| 476|
201011| 576|
201012| 326|
201101| 260|
201102| 219|
201103| 318|
201104| 240|
201105| 314|
201106| 329|
201107| 270|
+------+---------+
only showing top 20 rows

In [0]:
# Placed orders - 2 * Cancelled Orders
monthly_orders = monthly_placed_orders.join(monthly_cancelled_orders, monthly_placed_orders['YYYYMM'] == monthly_cancelled_orders['YYYYMM'], "inner").withColumn('Placed', col('PlacedBefore') - (2 * col('Cancelled'))).drop(monthly_placed_orders['YYYYMM'])
monthly_orders = monthly_orders.select('YYYYMM', 'Placed', 'Cancelled')
monthly_orders.show()

+------+------+---------+
YYYYMM|Placed|Cancelled|
+------+------+---------+
201103| 1347| 318|
201001| 1033| 300|
201002| 1489| 240|
201010| 2013| 476|
201009| 1633| 371|
201104| 1264| 240|
201106| 1354| 329|
201102| 955| 219|
201110| 1913| 362|
201008| 1331| 273|
201107| 1387| 270|
201012| 1373| 326|
201105| 1534| 314|
201109| 1661| 333|
201011| 2517| 576|
201007| 1329| 344|
201111| 2580| 441|
201108| 1181| 278|
201112| 723| 146|
201004| 1284| 304|
+------+------+---------+
only showing top 20 rows

#Monthly Sales

In [0]:
monthly_sales = dataframe.withColumn('YYYYMM', concat(col('invoice_date').substr(0,4), col('invoice_date').substr(6,2))).withColumn('Amount', col('quantity') * col('unit_price')).groupBy('YYYYMM').sum('Amount').orderBy('YYYYMM').withColumnRenamed('sum(Amount)', 'Amount')
monthly_sales = monthly_sales.withColumn('Amount', round(monthly_sales['Amount'], 2))
monthly_sales.show()

+------+----------+
YYYYMM| Amount|
+------+----------+
200912| 799847.11|
201001| 624032.89|
201002| 533091.43|
201003| 765848.76|
201004| 590580.43|
201005| 615322.83|
201006| 679786.61|
201007| 575236.36|
201008| 656776.34|
201009| 853650.43|
201010|1045168.35|
201011|1422654.64|
201012|1126445.47|
201101| 560000.26|
201102| 498062.65|
201103| 683267.08|
201104| 493207.12|
201105| 723333.51|
201106| 691123.12|
201107| 681300.11|
+------+----------+
only showing top 20 rows

#Monthly Active Users

In [0]:
monthly_active_users = dataframe.withColumn('YYYYMM', concat(col('invoice_date').substr(0,4), col('invoice_date').substr(6,2))).groupBy('YYYYMM', 'customer_id').count().groupBy('YYYYMM').count().withColumnRenamed('count', 'active_users')

monthly_active_users.show()

+------+------------+
YYYYMM|active_users|
+------+------------+
201103| 1021|
201001| 787|
201002| 808|
201010| 1578|
201009| 1203|
201104| 900|
201106| 1052|
201102| 799|
201110| 1426|
201008| 965|
201107| 994|
201012| 949|
201105| 1080|
201109| 1303|
201011| 1684|
201007| 989|
201111| 1712|
201108| 981|
201112| 687|
201004| 999|
+------+------------+
only showing top 20 rows

#New and Existing Users

In [0]:
with_yyyymm = dataframe.withColumn('YYYYMM', concat(col('invoice_date').substr(0,4), col('invoice_date').substr(6,2)).cast("Integer"))
monthly_new_users = with_yyyymm.groupBy('customer_id').min('YYYYMM').groupBy('min(YYYYMM)').count().orderBy('min(YYYYMM)').withColumnRenamed('count', 'new_users')
monthly_users = monthly_active_users.join(monthly_new_users, col('YYYYMM') == col('min(YYYYMM)'), "inner").withColumn('existing_users', col('active_users') - col('new_users'))
monthly_users = monthly_users.select('YYYYMM', 'new_users', 'existing_users').orderBy('YYYYMM')

monthly_users.show()

+------+---------+--------------+
YYYYMM|new_users|existing_users|
+------+---------+--------------+
200912| 1046| 0|
201001| 394| 393|
201002| 363| 445|
201003| 436| 676|
201004| 291| 708|
201005| 254| 809|
201006| 269| 827|
201007| 183| 806|
201008| 158| 807|
201009| 242| 961|
201010| 379| 1199|
201011| 322| 1362|
201012| 77| 872|
201101| 71| 713|
201102| 123| 676|
201103| 178| 843|
201104| 105| 795|
201105| 108| 972|
201106| 108| 944|
201107| 102| 892|
+------+---------+--------------+
only showing top 20 rows

#RFM Segmentation

In [0]:
rfm = dataframe
# Get frequency, monetary and recency
frequency = rfm.groupBy('customer_id', 'invoice_no').count().groupBy('customer_id').count().orderBy('customer_id').where(col('customer_id').isNotNull()).withColumnRenamed('count', 'Frequency')
monetary = rfm.withColumn('Amount', col('unit_price') * col('quantity')).groupBy('customer_id').sum('Amount').withColumnRenamed('sum(Amount)', 'monetary')
monetary = monetary.withColumn('monetary', round(monetary['monetary'], 2)).where(col('customer_id').isNotNull()).orderBy('customer_id')
# Weird way of finding recency cause it's difficult to convert the date to a timestamp and compare it with the current date...
recency = dataframe.withColumn('YYYYMM', concat(col('invoice_date').substr(0,4), col('invoice_date').substr(6,2)).cast("Integer")).groupBy('customer_id').max('YYYYMM')
recency = recency.withColumn('recency', 202110 - col('max(YYYYMM)') )

# Merge frequency, monetary and and recency
rfm = recency.join(frequency, frequency['customer_id'] == recency['customer_id'], "inner").drop(recency['customer_id']).drop('max(YYYYMM)')
rfm = rfm.join(monetary, monetary['customer_id'] == rfm['customer_id'], "inner").drop(rfm['customer_id'])
rfm.withColumnRenamed('Frequency', 'frequency').select('customer_id', 'recency','frequency','monetary')

rfm.show()

+-------+---------+-----------+--------+
recency|Frequency|customer_id|monetary|
+-------+---------+-----------+--------+
 1009| 6| 15447| 484.93|
 1107| 8| 18051| 2275.98|
 999| 15| 13623| 2446.36|
 998| 77| 17389|54587.03|
 1107| 3| 14832| -274.63|
 999| 15| 15727| 9445.51|
 1101| 5| 17753| 388.06|
 1000| 11| 17679| 3166.56|
 1000| 6| 16861| 902.37|
 999| 6| 13285| 3364.59|
 1002| 13| 16503| 3597.89|
 1198| 1| 13289| 307.95|
 1001| 3| 16574| 1301.78|
 1004| 7| 14450| 1128.44|
 1104| 1| 12799| 219.35|
 1099| 1| 15846| 107.01|
 999| 3| 15957| 930.1|
 999| 3| 13832| 602.44|
 1100| 1| 13840| 651.4|
 1000| 8| 17420| 1074.53|
+-------+---------+-----------+--------+
only showing top 20 rows